In [ ]:
# Run once for each radio

radio_number = 1 # change number for each radio up to 4

from pylab import *
from rtlsdr import *
from scipy import signal

sdr = RtlSdr()

# configure SDR device
N = 256*65536   # number of samples
sdr.sample_rate = 2.4e6
sdr.center_freq = 433.9e6
sdr.gain = 4

samples = sdr.read_samples(N)
sdr.close()

# use matplotlib to estimate and plot the PSD
plt.figure(figsize=(16,4))
psd(samples, NFFT=1024, Fs=sdr.sample_rate/1e6, Fc=sdr.center_freq/1e6)
xlabel('Frequency (MHz)')
ylabel('Relative power (dB)')
show()

# separating real and imaginary parts of the signal
real = np.real(samples)
imag = np.imag(samples)

# getting the amplitude
amplitude = real.copy()
for n in range(len(amplitude)):
    amplitude[n] = (real[n]**2+imag[n]**2)**(1/2)
    n = n + 1

# getting the time axis
t = np.arange(0,N/sdr.sample_rate,1/sdr.sample_rate)

# Signal Processing
# 1: filtering
# order 3 lowpass butterworth filter
b, a = signal.butter(3, 0.01)
filtered_signal = signal.filtfilt(b, a, amplitude)

# 2: normalization 0-1
# discard the initial part of the signal with huge amplitude
normalized_signal = (filtered_signal[4096:]-np.min(filtered_signal[4096:]))/np.ptp(filtered_signal[4096:])
lote = 65536
normalized_signal = np.array([])
cont = 0
iteracoes = int(len(amplitude)/lote)
for i in range(iteracoes):
    normalized_signal = np.append(normalized_signal,(amplitude[cont:cont+lote]-np.min(amplitude[cont:cont+lote]))/np.ptp(amplitude[cont:cont+lote]))
    cont = cont + lote
normalized_signal = normalized_signal[4096:]
    

# plot results
plt.figure(figsize=(16,4))
plt.plot(t[305000:445000], amplitude[305000:445000], 'b', alpha=0.75)
plt.plot(t[305000:445000], filtered_signal[305000:445000], 'r--')
plt.legend(('noisy signal controller '+ str(radio_number), 'order 3 lowpass butterworth'), loc='upper right')
plt.grid(True)
plt.show()

plt.figure(figsize=(16,4))
plt.plot(t[4096:], normalized_signal[:])
xlabel('Time (s)')
ylabel('Amplitude')
plt.grid()
show()

plt.figure(figsize=(16,4))
plt.plot(t[4096+459070:4096+594240], normalized_signal[459070:594240])
xlabel('Time (s)')
ylabel('Amplitude')
plt.grid()
show()

# reshape to 32x32
normalized_signal = normalized_signal.reshape(-1, 32, 32, 1)

# split train (75%) and test (25%) samples
train_samples = normalized_signal[:int(len(normalized_signal)*0.75)]
test_samples = normalized_signal[int(len(normalized_signal)*0.75):]

# creating labels for train and test samples
ones = np.ones((len(train_samples)))
zeros = np.zeros((len(train_samples)))

if radio_number == 1: train_labels = np.column_stack((ones,zeros,zeros,zeros))
if radio_number == 2: train_labels = np.column_stack((zeros,ones,zeros,zeros))
if radio_number == 3: train_labels = np.column_stack((zeros,zeros,ones,zeros))
if radio_number == 4: train_labels = np.column_stack((zeros,zeros,zeros,ones))
            
ones = np.ones((len(test_samples)))
zeros = np.zeros((len(test_samples)))

if radio_number == 1: test_labels = np.column_stack((ones,zeros,zeros,zeros))
if radio_number == 2: test_labels = np.column_stack((zeros,ones,zeros,zeros))
if radio_number == 3: test_labels = np.column_stack((zeros,zeros,ones,zeros))
if radio_number == 4: test_labels = np.column_stack((zeros,zeros,zeros,ones))            

# save to data
np.save("train_samples_radio"+str(radio_number)+".npy", train_samples)
np.save("train_labels_radio"+str(radio_number)+".npy", train_labels)

np.save("test_samples_radio"+str(radio_number)+".npy", test_samples)
np.save("test_labels_radio"+str(radio_number)+".npy", test_labels)